# Preparing a StatsBomb dataset

This notebook prepares a dataset from the [StatsBomb Open Data](https://github.com/statsbomb/open-data) repository. Specifically, it performs the folowing tasks.

1. Download the data.
2. Convert the raw event stream data to the [SPADL data format](https://socceraction.readthedocs.io/en/latest/documentation/spadl/spadl.html#spadl).
3. Compute a set of basic xG features.
4. Store the result in a HDF file.

**requirements**
- `socceraction==1.5.0`
- `soccerxg==1.0.0`

In [1]:
from pathlib import Path

import pandas as pd
from tqdm import tqdm

from socceraction.data.statsbomb import StatsBombLoader
from socceraction import spadl
from socceraction.spadl.statsbomb import convert_to_actions

In [2]:
from soccer_xg.data import HDFDataset
import soccer_xg.xg as xg
import soccer_xg.attributes as fs

/cw/dtaijupiter/NoCsBack/dtai/pieterr/Projects/soccer_xg/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuration

In [42]:
# Where the data should be stored
DATA_FP = Path("data")

# Which dataset to create (uncomment)
# DATASET = "spadl-statsbomb-bigfive-1516"
DATASET = "spadl-statsbomb-messi-biography"

In [12]:
(DATA_FP / DATASET).mkdir(parents=True, exist_ok=True)


## Set up the StatsBombLoader

In [13]:
SBL = StatsBombLoader(getter="remote", creds={"user": None, "passwd": None})

In [14]:
import warnings
# suppress warning about missing authentication while downloading public StatsBomb data
from statsbombpy.api_client import NoAuthWarning
warnings.simplefilter('ignore', NoAuthWarning)
# surpress warnings regarding data version
warnings.filterwarnings("ignore", message=".*fidelity.*")

In [15]:
# View all available competitions
competitions = SBL.competitions()
set(competitions.competition_name)

{'1. Bundesliga',
 'Champions League',
 'Copa del Rey',
 "FA Women's Super League",
 'FIFA U20 World Cup',
 'FIFA World Cup',
 'Indian Super league',
 'La Liga',
 'Liga Profesional',
 'Ligue 1',
 'NWSL',
 'North American League',
 'Premier League',
 'Serie A',
 'UEFA Euro',
 'UEFA Europa League',
 "UEFA Women's Euro",
 "Women's World Cup"}

In [16]:
if DATASET == "spadl-statsbomb-bigfive-1516":
    # Bigfive 15-16
    selected_competitions = competitions[
        competitions.competition_name.isin(['1. Bundesliga', 'La Liga', 'Premier League', 'Serie A', 'Ligue 1'])
        & (competitions.season_name == "2015/2016")
    ]
elif DATASET == "spadl-statsbomb-messi-biography":
    # Messi data
    selected_competitions = competitions[
        (competitions.competition_name == "La Liga")
        & (competitions.season_name.between("2004/2005", "2020/2021"))
    ]
else:
    raise ValueError(f"The dataset {DATASET} is not recognized.")
    
selected_competitions

,season_id,competition_id,competition_name,country_name,competition_gender,season_name
35,90,11,La Liga,Spain,male,2020/2021
36,42,11,La Liga,Spain,male,2019/2020
37,4,11,La Liga,Spain,male,2018/2019
38,1,11,La Liga,Spain,male,2017/2018
39,2,11,La Liga,Spain,male,2016/2017
40,27,11,La Liga,Spain,male,2015/2016
41,26,11,La Liga,Spain,male,2014/2015
42,25,11,La Liga,Spain,male,2013/2014
43,24,11,La Liga,Spain,male,2012/2013
44,23,11,La Liga,Spain,male,2011/2012


In [17]:
# Get games from all selected competitions
games = pd.concat([
    SBL.games(row.competition_id, row.season_id)
    for row in selected_competitions.itertuples()
])
games[["home_team_id", "away_team_id", "game_date", "home_score", "away_score"]]

,home_team_id,away_team_id,game_date,home_score,away_score
0,206,217,2020-10-31 21:00:00,1,1
1,1049,217,2021-01-09 18:30:00,0,4
2,217,209,2021-05-16 18:30:00,1,2
3,218,217,2021-02-07 21:00:00,2,3
4,422,217,2021-03-06 21:00:00,0,2
...,...,...,...,...,...
2,217,608,2005-05-01 19:00:00,2,0
3,217,221,2004-12-21 20:00:00,2,1
4,608,217,2004-12-11 20:00:00,1,2
5,217,216,2005-04-17 21:00:00,2,0


## Load and convert match data

In [18]:
# create a HDF dataset
dataset = HDFDataset(
    path=DATA_FP / DATASET / "dataset.h5", 
    mode="w"
)
for _, comp in selected_competitions.iterrows():
    # get name and id of competition
    competition_name, competition_id = comp.competition_name, comp.competition_id
    season_name, season_id = comp.season_name, comp.season_id
    print(f"Importing {competition_name} {season_name} ...")
    # import data
    dataset.import_data(
        SBL, 
        convert_to_actions, 
        competition_id, 
        season_id
    )
dataset.close()

Importing La Liga 2020/2021 ...


Loading game data...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [01:35<00:00,  2.74s/it]


Importing La Liga 2019/2020 ...


Loading game data...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [01:27<00:00,  2.66s/it]


Importing La Liga 2018/2019 ...


...

Loading game data...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [01:06<00:00,  2.57s/it]


Importing La Liga 2005/2006 ...


Loading game data...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:44<00:00,  2.60s/it]


Importing La Liga 2004/2005 ...


Loading game data...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:18<00:00,  2.61s/it]


## xG features

In [30]:
dataset = HDFDataset(
    path=DATA_FP / DATASET / "dataset.h5", 
    mode="a"
)

In [31]:
feature_generators = [
    fs.startlocation,
    fs.shot_angle,
    fs.shot_dist,
    fs.shot_bodypart_onehot
]

In [32]:
# Generate the features and labels for a single game as a test
GID = dataset.games().index[0]
X, y = fs.compute_attributes(
    game=dataset.games().loc[GID], 
    actions=dataset.actions(game_id=GID), 
    events=dataset.events(game_id=GID), 
    xfns=feature_generators
)
X.head()

,start_x_shot,start_y_shot,start_x_a1,start_y_a1,start_x_a2,start_y_a2,angle_shot,dist_shot,bodypart_foot_shot,bodypart_head_shot,bodypart_other_shot,bodypart_head/other_shot,bodypart_foot_left_shot,bodypart_foot_right_shot
action_id,,,,,,,,,,,,,,
151,94.54375,41.5225,102.94375,16.5325,102.94375,16.5325,0.623645,12.881039,True,False,False,False,True,False
207,96.99375,31.8325,79.75625,51.4675,79.75625,59.5425,0.264388,8.294462,True,False,False,False,False,True
240,103.11875,43.3075,101.63125,33.0225,92.18125,22.0575,1.371361,9.495718,True,False,False,False,True,False
360,89.03125,44.5825,81.06875,44.0725,81.06875,44.0725,0.585252,19.156990,True,False,False,False,True,False
431,95.59375,45.5175,91.21875,50.6175,77.65625,41.0975,0.885960,14.870452,True,False,False,False,False,True


In [33]:
# Now do it for all games in the dataset and store them
dataset["xg/features"], dataset["xg/labels"] = xg.prepare(dataset, xfns=feature_generators)

Preparing dataset: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1823/1823 [09:10<00:00,  3.31it/s]


In [34]:
dataset.close()

## Extract shots

We extract all shots (with their xG features and labels) from the dataset and store them in a Parquet file such that we can access them quickly.

In [43]:
dataset = HDFDataset(
    path=DATA_FP / DATASET / "dataset.h5", 
    mode="r"
)

In [44]:
# Load all shots with xG features and labels.
players = dataset.players().reset_index()[["player_id", "player_name"]].drop_duplicates(subset="player_id")
teams = dataset.teams().reset_index().drop_duplicates(subset="team_id")
games = (
    dataset.games().reset_index()
    .merge(teams.add_prefix('home_'), how='left')
    .merge(teams.add_prefix('away_'), how='left'))
shots = []
for game in tqdm(list(games.itertuples()), desc="Loading shots"):
    actions = dataset.actions(game.game_id).reset_index()
    actions = spadl.utils.add_names(actions)
    actions = spadl.utils.play_left_to_right(actions, game.home_team_id)
    shots.append(
        actions[actions.type_name.isin(['shot', 'shot_freekick', 'shot_penalty'])]
        .merge(players, how="left", on="player_id")
        .merge(teams, how="left", on="team_id")
        .assign(season=game.season_id)
        .assign(league=game.competition_id)
    )

Loading shots: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 867/867 [00:45<00:00, 19.03it/s]


In [45]:
df_shots = pd.concat(shots).set_index(["game_id", "action_id"])
print(f"Total shots: {len(df_shots)}")
# simplify bodypart
df_shots["bodypart_name_simple"] = df_shots.bodypart_name.copy()
df_shots.loc[df_shots.bodypart_name == "foot_left", "bodypart_name_simple"] = "foot"
df_shots.loc[df_shots.bodypart_name == "foot_right", "bodypart_name_simple"] = "foot"
# define filters
owngoal = df_shots.result_name == 'owngoal'
openplay = df_shots.type_name == 'shot'
# apply filters
df_shots_filt = df_shots.loc[~owngoal & openplay]
print(f"Filtered shots: {len(df_shots_filt)}")

Total shots: 21182
Filtered shots: 19614


In [46]:
df_shots_features = dataset["xg/features"].loc[df_shots_filt.index]
df_shots_labels = dataset["xg/labels"].loc[df_shots_filt.index]

In [47]:
# Save datasets
df_shots_filt.to_parquet(DATA_FP / DATASET / 'df_shots.parquet')
df_shots_features.to_parquet(DATA_FP / DATASET / 'df_shots_features.parquet')
df_shots_labels.to_parquet(DATA_FP / DATASET / 'df_shots_labels.parquet')

In [48]:
dataset.close()